# Alpha Vantage Sentiment Scores
## Sentiment Score Definition:
x <= -0.35: Bearish \
-0.35 < x <= -0.15: Somewhat-Bearish \
-0.15 < x < 0.15: Neutral \
0.15 <= x < 0.35: Somewhat_Bullish \
x >= 0.35: Bullish 

## Relevance Score Definition: 
0 < x <= 1, with a higher score indicating higher relevance.

In [4]:
# Importing Packages
import json
import pandas as pd

In [5]:
def process_json_to_df(json_data):
    # Extract the feed list from JSON
    feed = json_data['feed']
    
    # Create a list to store processed records
    processed_records = []
    
    for item in feed:
        # Create a record dictionary with desired fields
        record = {
            'title': item['title'],
            'url': item['url'],
            'time_published': item['time_published'],
            'summary': item['summary'],
            'source': item['source'],
            'topics': {topic['topic']: topic['relevance_score'] for topic in item['topics']},
            'overall_sentiment_score': item['overall_sentiment_score'],
            'overall_sentiment_label': item['overall_sentiment_label'],
            'ticker_sentiment': {
                ticker['ticker']: {
                    'relevance_score': ticker['relevance_score'],
                    'sentiment_score': ticker['ticker_sentiment_score'],
                    'sentiment_label': ticker['ticker_sentiment_label']
                } for ticker in item['ticker_sentiment']
            }
        }
        processed_records.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(processed_records)
    
    return df

In [10]:
ticker = "AEM"
json_file = f"news_sentiment_{ticker}.json"
with open(json_file) as f:
    data = json.load(f)
df = process_json_to_df(data)  
df.head()

,title,url,time_published,summary,source,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,3 American Century Investments Mutual Funds fo...,https://www.zacks.com/stock/news/2389783/3-ame...,20241231T101500,Consider investing in American Century Investm...,Zacks Commentary,"{'Life Sciences': '0.5', 'Energy & Transportat...",0.260052,Somewhat-Bullish,"{'ZIJMY': {'relevance_score': '0.075397', 'sen..."
1,Cartier Completes Private Placement,https://www.globenewswire.com/news-release/202...,20241230T214000,"VAL-D'OR, Quebec, Dec. 30, 2024 ( GLOBE NEWSWI...",GlobeNewswire,"{'Energy & Transportation': '1.0', 'Financial ...",0.269193,Somewhat-Bullish,"{'ECRFF': {'relevance_score': '0.204072', 'sen..."
2,Four Commodity Stocks To Watch In 2025 - ONGol...,https://www.benzinga.com/24/12/42719808/four-c...,20241230T120524,"Commodities are set for an interesting 2025, b...",Benzinga,"{'Mergers & Acquisitions': '0.310843', 'Energy...",0.196444,Somewhat-Bullish,"{'AEM': {'relevance_score': '0.127456', 'senti..."
3,Agnico Eagle Mines ( AEM ) Stock Moves -0.46...,https://www.zacks.com/stock/news/2389018/agnic...,20241227T225020,Agnico Eagle Mines (AEM) reachead $78.65 at th...,Zacks Commentary,"{'Earnings': '0.938793', 'Energy & Transportat...",0.181605,Somewhat-Bullish,"{'AEM': {'relevance_score': '0.532556', 'senti..."
4,IAMGOLD Stock Soars 102% YTD: Can the Momentum...,https://www.zacks.com/stock/news/2388988/iamgo...,20241227T200000,"The IAG stock performs well YTD, driven by sol...",Zacks Commentary,"{'Economy - Monetary': '0.451494', 'Financial ...",0.351766,Bullish,"{'AEM': {'relevance_score': '0.122477', 'senti..."
